In [1]:
import findspark

In [2]:
# /opt/manual/spark: this is SPARK_HOME path
findspark.init("/opt/manual/spark")

In [3]:
from pyspark.sql import SparkSession, functions as F

In [4]:
spark = SparkSession.builder \
.appName("Datetime Extract New Features") \
.master("local[2]") \
.getOrCreate()

2022-08-27 19:47:41,332 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
# Data source: https://data.sfgov.org/Public-Safety/Fire-Incidents/wr8u-xric/data

In [6]:
# ! wget -P ~/datasets \
# https://github.com/erkansirin78/datasets/raw/master/Fire_Incidents.csv.gz

In [7]:
! ls -l ~/datasets | grep Fire

-rw-rw-r--. 1 train train 41002480 Aug 27 19:45 Fire_Incidents.csv.gz


In [8]:
df = spark.read.option("header", True) \
.option("inferSchema", True) \
.option("header", True) \
.option("compression","gzip") \
.csv("file:///home/train/datasets/Fire_Incidents.csv.gz")

In [9]:
ts_cols = ['Incident Date','Alarm DtTm']

In [10]:
# Create new dataframe which includes only date time columns
df2 = df.select(ts_cols)

In [11]:
df2.dtypes

[('Incident Date', 'string'), ('Alarm DtTm', 'string')]

In [12]:
# First we need date and timestamp columns

In [13]:
df3 = df2.withColumn("Incident Date", 
                     F.to_date(F.col("Incident Date"), 'MM/dd/yyyy')) \
.withColumn("Alarm DtTm", 
                     F.to_timestamp(F.col("Alarm DtTm"), 'MM/dd/yyyy hh:mm:ss a'))
df3.limit(3).toPandas()

Incident Date          Alarm DtTm
0    2018-06-05 2018-06-05 18:38:01
1    2019-08-29 2019-08-29 20:09:25
2    2018-06-14 2018-06-14 20:37:56

In [14]:
df3.printSchema()

root
 |-- Incident Date: date (nullable = true)
 |-- Alarm DtTm: timestamp (nullable = true)



## Get year

In [15]:
df4 = df3.withColumn("Alarm_DtTm_Year", 
                     F.year(F.col("Alarm DtTm")))
df4.limit(3).toPandas()

Incident Date          Alarm DtTm  Alarm_DtTm_Year
0    2018-06-05 2018-06-05 18:38:01             2018
1    2019-08-29 2019-08-29 20:09:25             2019
2    2018-06-14 2018-06-14 20:37:56             2018

In [16]:
df3.select(F.year("Alarm DtTm").alias("Alarm_DtTm_Year")).show(3)

+---------------+
|Alarm_DtTm_Year|
+---------------+
|           2018|
|           2019|
|           2018|
+---------------+
only showing top 3 rows



## Get month

In [17]:
df4 = df3.withColumn("Alarm_DtTm_Month", 
                     F.month(F.col("Alarm DtTm")))
df4.limit(3).toPandas()

Incident Date          Alarm DtTm  Alarm_DtTm_Month
0    2018-06-05 2018-06-05 18:38:01                 6
1    2019-08-29 2019-08-29 20:09:25                 8
2    2018-06-14 2018-06-14 20:37:56                 6

In [18]:
df3.select(F.month("Alarm DtTm")).show(3)

+-----------------+
|month(Alarm DtTm)|
+-----------------+
|                6|
|                8|
|                6|
+-----------------+
only showing top 3 rows



## Get week of year

In [19]:
df4 = df3.withColumn("Alarm_DtTm_Week", 
                     F.weekofyear(F.col("Alarm DtTm")))
df4.limit(3).toPandas()

Incident Date          Alarm DtTm  Alarm_DtTm_Week
0    2018-06-05 2018-06-05 18:38:01               23
1    2019-08-29 2019-08-29 20:09:25               35
2    2018-06-14 2018-06-14 20:37:56               24

In [20]:
df3.select(F.weekofyear("Alarm DtTm")).show(3)

+----------------------+
|weekofyear(Alarm DtTm)|
+----------------------+
|                    23|
|                    35|
|                    24|
+----------------------+
only showing top 3 rows



## Get day of month

In [21]:
df4 = df3.withColumn("Alarm_DtTm_DayofMonth", 
                     F.dayofmonth(F.col("Alarm DtTm")))
df4.limit(3).toPandas()

Incident Date          Alarm DtTm  Alarm_DtTm_DayofMonth
0    2018-06-05 2018-06-05 18:38:01                      5
1    2019-08-29 2019-08-29 20:09:25                     29
2    2018-06-14 2018-06-14 20:37:56                     14

## Get day of week

In [22]:
df4 = df3.withColumn("Alarm_DtTm_DayofWeek", 
                     F.dayofweek(F.col("Alarm DtTm"))) 
df4.limit(3).toPandas()

Incident Date          Alarm DtTm  Alarm_DtTm_DayofWeek
0    2018-06-05 2018-06-05 18:38:01                     3
1    2019-08-29 2019-08-29 20:09:25                     5
2    2018-06-14 2018-06-14 20:37:56                     5

In [23]:
# Min max of days
df3.withColumn("Alarm_DtTm_DayofWeek", 
                     F.dayofweek(F.col("Alarm DtTm"))) \
.selectExpr("MIN(Alarm_DtTm_DayofWeek)", "MAX(Alarm_DtTm_DayofWeek)") \
.limit(3).toPandas()

min(Alarm_DtTm_DayofWeek)  max(Alarm_DtTm_DayofWeek)
0                          1                          7

# Get day name

In [24]:
df4 = df3.withColumn("Alarm_DtTm_DayName", 
                     F.date_format(F.col("Alarm DtTm"), 'E'))
df4.limit(3).toPandas()

Incident Date          Alarm DtTm Alarm_DtTm_DayName
0    2018-06-05 2018-06-05 18:38:01                Tue
1    2019-08-29 2019-08-29 20:09:25                Thu
2    2018-06-14 2018-06-14 20:37:56                Thu

# Get day names other than English

In [25]:
from pyspark.sql.types import StringType

In [26]:
# write a function that converts day number to day name
def switch_tr_day(day_index):
    my_dict = {
        1: 'Pazartesi',
        2: 'Salı',
        3: 'Çarşamba',
        4: 'Perşembe',
        5: 'Cuma',
        6: 'Cumartesi',
        7: 'Pazar'
    }
    
    return my_dict.get(day_index)

In [27]:
switch_tr_day(4)

'Perşembe'

In [29]:
switch_day_func = F.udf(lambda z: switch_tr_day(z), StringType())

In [30]:
# Use your registered function

df5 = df4 \
.withColumn("Alarm_DtTm_DayofWeek", F.dayofweek(F.col("Alarm DtTm")))


df5.select(switch_day_func('Alarm_DtTm_DayofWeek'), "Alarm_DtTm_DayofWeek").limit(5).toPandas()

<lambda>(Alarm_DtTm_DayofWeek)  Alarm_DtTm_DayofWeek
0                       Çarşamba                     3
1                           Cuma                     5
2                           Cuma                     5
3                      Cumartesi                     6
4                           Cuma                     5

Traceback (most recent call last):
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


In [31]:
spark.stop()